In [1]:
%cd Hate_speech_task/Extra_data_offense_osacl/

/home/ashapiro/Hate_speech_task/Extra_data_offense_osacl


In [2]:
from utils import getData
from datasets import load_dataset 
import pandas as pd 
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import pandas as pd 
import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import Counter
from datasets import load_metric
from transformers import EarlyStoppingCallback

In [4]:
f1 = load_metric("f1")
recall = load_metric("recall")
precision =  load_metric("precision")
def preprocess_function(examples, tok):
    return tok(examples["text"], truncation=True, max_length=512)


def compute_metrics(p):    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    metric = f1.compute(predictions=predictions, references=labels, average="macro")
    metric.update(recall.compute(predictions=predictions, references=labels, average="macro"))
    metric.update(precision.compute(predictions=predictions, references=labels, average="macro"))
    return metric
data = getData(sub_task = f"A", return_type = "dataset", pre_proccessed = True)
tokenizer = AutoTokenizer.from_pretrained("/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_data = data.map(preprocess_function,fn_kwargs = {'tok':tokenizer}, batched=True)

Pre-Processed


100%|██████████| 2/2 [00:00<00:00, 600.73it/s]


In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 10552
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1270
    })
})

In [6]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/", num_labels=2)

In [7]:
training_args = TrainingArguments(
    output_dir="./Checkpoints/MarBertv2_hyperparams",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    group_by_length = True, 
    report_to = 'none'
    )
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

loading configuration file /scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/config.json
Model config BertConfig {
  "_name_or_path": "/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file /scr

In [8]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
        "weight_decay" : trial.suggest_float("weight_decay", 1e-10, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 20),
        "seed": trial.suggest_int("seed", 1, 50),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32,64]),
    }

In [9]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize",  hp_space=my_hp_space)

[I 2022-03-28 21:05:46,609] A new study created in memory with name: no-name-6fd228a3-ff94-4226-a56d-73957c3a978b
Trial:
loading configuration file /scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/config.json
Model config BertConfig {
  "_name_or_path": "/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transfor

{'loss': 0.7666, 'learning_rate': 0.0006734533661655461, 'epoch': 0.38}


  6%|▌         | 1003/17147 [00:59<16:06, 16.71it/s]

{'loss': 0.7958, 'learning_rate': 0.000653225896766689, 'epoch': 0.76}


  8%|▊         | 1319/17147 [01:17<15:26, 17.09it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4

 99%|█████████▊| 314/318 [00:03<00:00, 80.58it/s]/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                    
100%|██████████| 318/318 [00:03<00:00, 80.58it/s]
                                                 Saving model checkpoint to ./Checkpoints/MarBertv2_hyperparams/run-0/checkp

...

{'loss': 0.6932, 'learning_rate': 9.31066598879098e-07, 'epoch': 15.91}


100%|██████████| 10560/10560 [13:42<00:00, 16.00it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4

100%|██████████| 318/318 [00:03<00:00, 87.00it/s]/cm/shared/ebtree/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

                                                     
100%|██████████| 10560/10560 [13:46<00:00, 16.00it/s]
                                                 Saving model checkpoint to ./Checkpoints/MarBertv2_hyperparams/run-9

{'eval_loss': 0.6922903060913086, 'eval_f1': 0.4054307116104869, 'eval_recall': 0.5, 'eval_precision': 0.3409448818897638, 'eval_runtime': 3.7462, 'eval_samples_per_second': 339.012, 'eval_steps_per_second': 84.886, 'epoch': 16.0}


Model weights saved in ./Checkpoints/MarBertv2_hyperparams/run-9/checkpoint-10560/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/MarBertv2_hyperparams/run-9/checkpoint-10560/tokenizer_config.json
Special tokens file saved in ./Checkpoints/MarBertv2_hyperparams/run-9/checkpoint-10560/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./Checkpoints/MarBertv2_hyperparams/run-9/checkpoint-660 (score: 0.4054307116104869).
100%|██████████| 10560/10560 [13:53<00:00, 12.66it/s]
[I 2022-03-28 22:41:56,071] Trial 9 finished with value: 1.2463755935002507 and parameters: {'learning_rate': 0.00016386772140272125, 'weight_decay': 3.0516285044005945e-10, 'num_train_epochs': 16, 'seed': 1, 'per_device_train_batch_size': 16}. Best is trial 3 with value: 2.52298489012904.


{'train_runtime': 833.8107, 'train_samples_per_second': 202.482, 'train_steps_per_second': 12.665, 'train_loss': 0.6966054360071818, 'epoch': 16.0}
